In [1]:
#加载模块
import time
import copy
import os
from PIL import ImageFile
import torch
import torchvision
ImageFile.LOAD_TRUNCATED_IMAGES = True
from fine_tuning_config_file import *
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np

In [2]:
#采用gpu训练模型
use_gpu = GPU_MODE
if use_gpu:
    torch.cuda.set_device(CUDA_DEVICE)

In [3]:
#数据转换，为了扩增数据
data_transforms = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
#加载数据集
data_dir = 'WCE'
dsets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

In [17]:
#训练网络模型
def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()
                mode='val'

            running_loss = 0.0
            running_corrects = 0
            TP=0
            TN=0
            FN=0
            FP=0
            TP_item=0
            TN_item=0
            FN_item=0
            FP_item=0
            counter=0
            # Iterate over data.
            for data in dset_loaders[phase]:
                inputs, labels = data
                #print(inputs.size())
                # wrap them in Variable
                inputs = torch.autograd.Variable(inputs.float().cuda())                            
                labels = torch.autograd.Variable(labels.long().cuda())

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                #------------保存文件夹--------------
                #preds=preds.cpu().numpy()
                #0是abnormal，1是normal
               # print(preds)
                if counter==0:
                    phase_pred=preds.cpu().numpy()
                    phase_label=labels.cpu().numpy()
                else:
                    phase_pred = np.append(phase_pred, preds.cpu().numpy())
                    phase_label = np.append(phase_label,labels.cpu().numpy())
                # TP    predict 和 label 同时为1
                TP += ((preds == 1) & (labels.data == 1)).cpu().sum()
                    #TP_item +=torch.sum((preds[i] == 1) & (labels.data[i] == 1))
                    #print(TP_item)
                    # TN    predict 和 label 同时为0
                TN += ((preds == 0) & (labels.data == 0)).cpu().sum()
                    #print(TN_item)
                    # FN    predict 0 label 1
                FN += ((preds == 0) & (labels.data == 1)).cpu().sum()
                    # FP    predict 1 label 0
                FP += ((preds == 1) & (labels.data == 0)).cpu().sum()
                loss = criterion(outputs, labels)
                # print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                # if counter%10==0:
                #     print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    # print('loss backward')
                    loss.backward()
                    # print('done loss backward')
                    optimizer.step()
                    # print('done optim')
                # print evaluation statistics
                try:
                    # running_loss += loss.data[0]
                    running_loss += loss.item()
                    # print(labels.data)
                    # print(preds)
                    running_corrects += torch.sum(preds == labels.data)
                    """TP+=TP_item
                    TN+=TN_item
                    FN+=FN_item
                    FP+=FP_item"""
                    # print('running correct =',running_corrects)
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            print('trying epoch loss')
            print("TP:",TP)
            print("TN:",TN)
            print("FN:",FN)
            print("FP:",FP)
            r = TP.float() / (TP + FN).float()
            TNR= TN .float()/ (FP + TN).float()
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects.item() / float(dset_sizes[phase])  
            epoch_auc = roc_auc_score(phase_label, phase_pred)
            print('{} Loss: {:.4f} Acc: {:.4f} Auc: {:.4f} recall: {:.4f} specificity: {:.4f} '.format(
                phase, epoch_loss, epoch_acc,epoch_auc,r,TNR))


            # deep copy the model
            if phase == 'val':
                """if USE_TENSORBOARD:
                    foo.add_scalar_value('epoch_loss',epoch_loss,step=epoch)
                    foo.add_scalar_value('epoch_acc',epoch_acc,step=epoch)"""
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy = ',best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')
    return best_model

In [18]:
#用迁移学习搭建自己的网络
class BCNN(torch.nn.Module):
    """B-CNN for CUB200.

    The B-CNN model is illustrated as follows.
    conv1^2 (64) -> pool1 -> conv2^2 (128) -> pool2 -> conv3^3 (256) -> pool3
    -> conv4^3 (512) -> pool4 -> conv5^3 (512) -> bilinear pooling
    -> sqrt-normalize -> L2-normalize -> fc (200).
    The network accepts a 3*448*448 input, and the pool5 activation has shape
    512*28*28 since we down-sample 5 times.
    vgg16：conv1^2 (64) -> pool1 -> conv2^2 (128) -> pool2 -> conv3^3 (256) -> pool3
    -> conv4^3 (512) -> pool4 -> conv5^3 (512) 
    Attributes:
        features, torch.nn.Module: Convolution and pooling layers.
        fc, torch.nn.Module: 200.
    """
    def __init__(self):
        """Declare all needed layers."""
        torch.nn.Module.__init__(self)
        # Convolution and pooling layers of VGG-16.
        self.features = torchvision.models.vgg16(pretrained=False,progress=True).features#获取vgg16的特征
        self.features = torch.nn.Sequential(*list(self.features.children())#获取零到最后第二层的特征
                                            [:-1])# Remove pool5.
        #self.features2 = torch.nn.Sequential(*list(self.features.children())#获取零到最后第二层的特征
          #                                  [:-])
        
        # Linear classifier.
        self.fc = torch.nn.Linear(512**2, 2)#定义一个输入特征为512*2，输出特征为200的线性层
        #self.bn = torch.nn.BatchNorm1d(512**2)#加入
        for param in self.features.parameters():
            param.requires_grad = False


    def forward(self, X):
        """Forward pass of the network.

        Args:
            X, torch.autograd.Variable of shape N*3*448*448.

        Returns:
            Score, torch.autograd.Variable of shape N*200.
        """
        N = X.size()[0]#n为x的第一维形状
        assert X.size() == (N, 3, 224, 224)#判断x的形状是否为（n，3，448，448）
        X = self.features(X)#？？
        assert X.size() == (N, 512, 14, 14)#判断x的形状是否为（n，512，28，28）
        X = X.view(N, 512, 14**2)#转换x的形状为(n，512，28*28)
        X = torch.bmm(X, torch.transpose(X, 1, 2)) / (14**2)  # Bilinear  x矩阵乘x的转置然后除28*28
        assert X.size() == (N, 512, 512)#判断x的形状是否为
        X = X.view(N, 512**2)
        X = torch.sqrt(X + 1e-5)#开根号
        X = torch.nn.functional.normalize(X)#进行参数规范化
        #X = self.bn(X)
        X = self.fc(X)#加入线性层，输出为（n，200）
        assert X.size() == (N, 2)
        return X

In [19]:
#改变训练模型的学习率
def exp_lr_scheduler(optimizer, epoch, init_lr=BASE_LR, lr_decay_epoch=EPOCH_DECAY):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (DECAY_WEIGHT**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [24]:
model_ft =BCNN().cuda()

criterion = torch.nn.CrossEntropyLoss()
#criterion = FocalLoss(gamma=0)

if use_gpu:
    criterion=criterion.cuda()
    model_ft = model_ft.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

# Save model
model_ft.save_state_dict('fine_tuned_best_model.pt')

Epoch 0/29
----------
LR is set to 0.001
trying epoch loss
TP: tensor(9780)
TN: tensor(6386)
FN: tensor(1220)
FP: tensor(2054)
train Loss: 0.0389 Acc: 0.8316 Auc: 0.8229 recall: 0.8891 specificity: 0.7566 
trying epoch loss
TP: tensor(4502)
TN: tensor(3030)
FN: tensor(357)
FP: tensor(810)
val Loss: 0.0321 Acc: 0.8658 Auc: 0.8578 recall: 0.9265 specificity: 0.7891 
new best accuracy =  0.8658466490401195
Epoch 1/29
----------


KeyboardInterrupt: 

In [13]:
model_ft =BCNN().cuda()
model_ft.load_state_dict(torch.load('models/model_ft_vgg16.pth'),strict=False)
criterion = torch.nn.CrossEntropyLoss()
#criterion = FocalLoss(gamma=0)

if use_gpu:
    criterion=criterion.cuda()
    model_ft = model_ft.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

Epoch 0/29
----------
LR is set to 0.001
trying epoch loss
TP: tensor(9796)
TN: tensor(6373)
FN: tensor(1204)
FP: tensor(2067)
train Loss: 0.0386 Acc: 0.8317 Auc: 0.8228 recall: 0.8905 specificity: 0.7551 
trying epoch loss
TP: tensor(4477)
TN: tensor(3088)
FN: tensor(382)
FP: tensor(752)
val Loss: 0.0317 Acc: 0.8696 Auc: 0.8628 recall: 0.9214 specificity: 0.8042 
new best accuracy =  0.869640188527417
Epoch 1/29
----------
trying epoch loss
TP: tensor(9924)
TN: tensor(6690)
FN: tensor(1076)
FP: tensor(1750)
train Loss: 0.0347 Acc: 0.8546 Auc: 0.8474 recall: 0.9022 specificity: 0.7927 
trying epoch loss
TP: tensor(4698)
TN: tensor(2735)
FN: tensor(161)
FP: tensor(1105)
val Loss: 0.0335 Acc: 0.8545 Auc: 0.8396 recall: 0.9669 specificity: 0.7122 
Epoch 2/29
----------


KeyboardInterrupt: 

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchsummary import summary
summary(BCNN().cuda(), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_ft_vgg =torchvision.models.vgg16(pretrained=True).to(device)
num_ftrs = model_ft_vgg.classifier[0].in_features
feature_model = list(model_ft_vgg.classifier[:-6].children())
feature_model.pop()            
feature_model.append(torch.nn.Linear(num_ftrs, NUM_CLASSES))
model_ft_vgg.classifier = torch.nn.Sequential(*feature_model).to(device)
from torchsummary import summary
summary(model_ft_vgg, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_ft_vgg =torchvision.models.vgg16(pretrained=True).to(device)
num_ftrs = model_ft_vgg.classifier[0].in_features
feature_model = list(model_ft_vgg.classifier[:-6].children())
feature_model.pop()            
feature_model.append(torch.nn.Linear(num_ftrs, NUM_CLASSES))
model_ft_vgg.classifier = torch.nn.Sequential(*feature_model).to(device)
for param in model_ft_vgg.features.parameters():
    param.requires_grad = False

criterion = torch.nn.CrossEntropyLoss()
#criterion = FocalLoss(gamma=0)

if use_gpu:
    criterion=criterion.cuda()
    model_ft_vgg = model_ft_vgg.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft_vgg.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft_vgg, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

# Save model
model_ft.save_state_dict('models/fine_tuned_best_model.pth')

Epoch 0/29
----------
LR is set to 0.001
trying epoch loss
TP: tensor(8828)
TN: tensor(6190)
FN: tensor(2172)
FP: tensor(2250)
train Loss: 0.1265 Acc: 0.7725 Auc: 0.7680 recall: 0.8025 specificity: 0.7334 
trying epoch loss
TP: tensor(4272)
TN: tensor(2958)
FN: tensor(587)
FP: tensor(882)
val Loss: 0.0980 Acc: 0.8311 Auc: 0.8248 recall: 0.8792 specificity: 0.7703 
new best accuracy =  0.8311300149442464
Epoch 1/29
----------
trying epoch loss
TP: tensor(9018)
TN: tensor(6414)
FN: tensor(1982)
FP: tensor(2026)
train Loss: 0.1208 Acc: 0.7938 Auc: 0.7899 recall: 0.8198 specificity: 0.7600 
trying epoch loss
TP: tensor(4546)
TN: tensor(2716)
FN: tensor(313)
FP: tensor(1124)
val Loss: 0.1201 Acc: 0.8348 Auc: 0.8214 recall: 0.9356 specificity: 0.7073 
new best accuracy =  0.8348085986895045
Epoch 2/29
----------
trying epoch loss
TP: tensor(9088)
TN: tensor(6455)
FN: tensor(1912)
FP: tensor(1985)
train Loss: 0.1219 Acc: 0.7995 Auc: 0.7955 recall: 0.8262 specificity: 0.7648 
trying epoch loss

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_ft_vgg =torchvision.models.vgg16(pretrained=True).to(device)
num_ftrs = model_ft_vgg.classifier[0].in_features
feature_model = list(model_ft_vgg.classifier[:-6].children())
feature_model.pop()            
feature_model.append(torch.nn.Linear(num_ftrs, NUM_CLASSES))
model_ft_vgg.classifier = torch.nn.Sequential(*feature_model).to(device)
for param in model_ft_vgg.features.parameters():
    param.requires_grad = True

criterion = torch.nn.CrossEntropyLoss()
#criterion = FocalLoss(gamma=0)

if use_gpu:
    criterion=criterion.cuda()
    model_ft_vgg = model_ft_vgg.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft_vgg.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft_vgg, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

# Save model
torch.save(model_ft.state_dict(), 'models/fine_tuned_best_model.pth')

Epoch 0/29
----------
LR is set to 0.001


KeyboardInterrupt: 

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
    
class FocalLoss(nn.Module):
    def __init__(self, gamma=0.25, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()